#### Fetching Microsoft Stock Data (official code example from yfinance)

In [3]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

# get historical market data
hist = msft.history(period="1mo")

# show meta information about the history (requires history() to be called first)
msft.history_metadata

# show actions (dividends, splits, capital gains)
msft.actions
msft.dividends
msft.splits
msft.capital_gains  # only for mutual funds & etfs

# show share count
msft.get_shares_full(start="2022-01-01", end=None)

# show financials:
# - income statement
msft.income_stmt
msft.quarterly_income_stmt
# - balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet
# - cash flow statement
msft.cashflow
msft.quarterly_cashflow
# see `Ticker.get_income_stmt()` for more options

# show holders
msft.major_holders
msft.institutional_holders
msft.mutualfund_holders
msft.insider_transactions
msft.insider_purchases
msft.insider_roster_holders

# show recommendations
msft.recommendations
msft.recommendations_summary
msft.upgrades_downgrades

# Show future and historic earnings dates, returns at most next 4 quarters and last 8 quarters by default.
# Note: If more are needed use msft.get_earnings_dates(limit=XX) with increased limit argument.
msft.earnings_dates

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

# show options expirations
msft.options

# show news
msft.news

# get option chain for specific expiration
opt = msft.option_chain('2024-06-21')
# data available via: opt.calls, opt.puts

#### Example: Data for Google Stock 

In [1]:
import yfinance as yf
from src.utils import display_df

# Get stock data for Apple (AAPL)
ticker = yf.Ticker("GOOGL")

# Get historical market data
# hist = ticker.history(period="max")  # Get all available data
# or
hist = ticker.history(period="6mo")  # Get data for the past year

# Display the data
display_df(hist)


| Open    | High    | Low     | Close   | Volume      | Dividends   | Stock Splits   |
|:--------|:--------|:--------|:--------|:------------|:------------|:---------------|
| 136.683 | 137.312 | 135.924 | 136.493 | 2.54768e+07 | 0           | 0              |
| 138.811 | 141.538 | 137.912 | 138.181 | 4.91072e+07 | 0           | 0              |
| 139.33  | 140.529 | 139.02  | 140.259 | 2.74883e+07 | 0           | 0              |
| 140.609 | 141.827 | 140.549 | 141.328 | 2.65146e+07 | 0           | 0              |
| 141.428 | 142.516 | 141.028 | 141.358 | 1.67803e+07 | 0           | 0              |


#### Preparation for saving to Postgres

In [9]:
import yfinance as yf

symbol = "AAPL"
period = "1y"

# Get stock data for Apple (AAPL)
ticker = yf.Ticker(symbol)
hist = ticker.history(period=period)


In [3]:
from confluent_kafka import Producer
import yfinance as yf
import json
import time

# Callback function to handle delivery reports
def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

def fetch_and_send_stock_data(symbol, period="1d", max_retries=2):
    # Fetch stock data
    stock = yf.Ticker(symbol)
    hist = stock.history(period=period)  # Fetch data for the specified period

    # Kafka configuration
    conf = {'bootstrap.servers': "localhost:9094"}
    producer = Producer(**conf)
    topic = 'stock_data'

    # Iterate over each row in the DataFrame
    for index, row in hist.iterrows():
        # Prepare data for sending to Kafka
        data = row.to_dict()
        data['symbol'] = symbol  # Add the symbol to the data
        data['date'] = index.strftime('%Y-%m-%d')  # Add the date to the data

        # Attempt to send data to Kafka with retries
        for attempt in range(max_retries):
            try:
                print(str(data))
                producer.produce(topic, json.dumps(data).encode('utf-8'), callback=delivery_report)
                producer.flush()
                break  # Exit the retry loop on success
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)  # Exponential backoff
                else:
                    print("Max retries reached. Failed to send data to Kafka.")

# Example usage for a longer period, e.g., 5 days
fetch_and_send_stock_data("AAPL", "5d")

{'Open': 207.3699951171875, 'High': 220.1999969482422, 'Low': 206.89999389648438, 'Close': 213.07000732421875, 'Volume': 198134300.0, 'Dividends': 0.0, 'Stock Splits': 0.0, 'symbol': 'AAPL', 'date': '2024-06-12'}
Message delivered to stock_data [0]
{'Open': 214.74000549316406, 'High': 216.75, 'Low': 211.60000610351562, 'Close': 214.24000549316406, 'Volume': 97862700.0, 'Dividends': 0.0, 'Stock Splits': 0.0, 'symbol': 'AAPL', 'date': '2024-06-13'}
Message delivered to stock_data [0]
{'Open': 213.85000610351562, 'High': 215.1699981689453, 'Low': 211.3000030517578, 'Close': 212.49000549316406, 'Volume': 70122700.0, 'Dividends': 0.0, 'Stock Splits': 0.0, 'symbol': 'AAPL', 'date': '2024-06-14'}
Message delivered to stock_data [0]
{'Open': 213.3699951171875, 'High': 218.9499969482422, 'Low': 212.72000122070312, 'Close': 216.6699981689453, 'Volume': 93728300.0, 'Dividends': 0.0, 'Stock Splits': 0.0, 'symbol': 'AAPL', 'date': '2024-06-17'}
Message delivered to stock_data [0]
{'Open': 217.58999